# 1. Introduction

We will be buidling a library that performs Automatic Differentiation (AD). Any client can install and use the library for personal or professional use and obtain an estimate of the value of the derivative (or gradient / Jacobian in higher dimension) of the function provided at the data point given as argument.

Performing fast differentiation and obtaining derivatives is absolutely necessary as it is a skill needed for a lot of real life applications. Indeed, most of systems modeling use differential equations to describe a behaviour and these equations require to take derivatives of sometimes complex function. Also, taking the gradient of a function at a given point and cancel it is the most effective way (at least analytically) to find the extrema of a function. Computing the values of extrema is a key feature to optimize a function and the processes it represents.



- 

# 2. Background 

We will provide a brief background to motivate our implementation of Automatic Differentiaion.

### 1. Intro to AD

AD is a way to obtain the value of the derivative of a function $f$ at a point $X$. The objective is to obtain a method providing more precise values than the naive estimators using Taylor expansion. Such estimators require fine tuning of parameters in order to give an approximation which is close enough to the truth value but which does not fail because of the floating point approximation.

### 2. Chain Rule

The Chain Rule is the key element of AD. Indeed we can decompose recursively a function $f$ into elementary components. For example, if we consider the function $f(x, y) = cos(x+y) \times sin(x-y)$, we can write it $f(x,y) = prod(cos(sum(x,y)), sin(difference(x, y))))$. Although unclear for a human eye, such a function is easier to derive by a machine using the chain rule:

$\frac{\partial f}{\partial x} = \frac{\partial f}{\partial u}\frac{\partial u}{\partial x} + \frac{\partial f}{\partial v}\frac{\partial v}{\partial x}$

In other words, you can compute the derivative of a function with respect to a variable by computing recursively the derivatives of each of the components and the derivative of the main function with respect to its components.

### Evaluation graph

When we can write a function as a series of simple components, we can obtain its evaluation graph. Here would be the evaluation graph for the example function provided above. 



![alt](evaluation_graph.PNG)

We also have the following evaluation table

|trace|elem operation|value of the function (as a function of $(x,y)$)|elem derivative|$\nabla_x$|$\nabla_y$|
|--|--|--|--|--|--|
|$u_{-1}$|$u_{-1}$|$x$|$\dot{u}_{-1}$|$1$|$0$|
|$u_0$|$u_0$|$y$|$\dot{u}_0$|$0$|$1$|
|$u_1$|$u_{-1} + u_0$|$x+y$|$\dot{u}_{-1} + \dot{u}_0$|$1$|$1$|
|$u_2$|$u_{-1} - u_0$|$x-y$|$\dot{u}_{-1} - \dot{u}_0$|$1$|$-1$|
|$u_3$|$cos(u_1)$|$cos(x+y)$|$-\dot{u}_1sin(u_1)$|$-sin(x+y)$|$-sin(x+y)$|
|$u_4$|$sin(u_2)$|$sin(x-y)$|$\dot{u}_2cos(u_2)$|$cos(x-y)$|$-cos(x-y)$|
|$u_5$|$u_3u_4$|$cos(x+y)sin(x-y)$|$\dot{u}_3u_4+u_3\dot{u}_4$|$-sin(x+y)sin(x-y) + cos(x+y)cos(x-y)$|$-sin(x+y)sin(x-y)-cos(x+y)cos(x-y)$|

- 

# 3. How to Use GrADim

We will briefly demonstrate how to use our package

### Installing and importing the package

A user can install the package using:

In [ ]:
>>> pip install GrADim

The package can be imported using the following command:

In [ ]:
>>> import GrADim as ad

### Using the package

An instance of AD object can be created and used to find the derivative for the required point:

In [ ]:
>>> import GrADim as ad
>>> import numpy as np

>>> def fun(x,y):
    return np.cos(x+y)*np.cos(x-y)

>>> autodiff = ad.derivative(fun)
>>> autodiff.fwd_pass((1,1))
>>> autodiff.rev_pass()

# 4. Software Organization 

1. Directory Structure
    The package directory would look like the following tree.

    File README.md will contain instructions and provide examples using the package. License folder will contain relevant licensing information. File requirements.txt will contain details for package to be distributed.




In [ ]:
master
├── LICENSE
├── README.md     
├── docs
│   ├── ...
│   └── ...
├── requirements.txt
├── travis.yml
├── GrADim
│   ├── ...
│   └── ...
├── Tests
    ├── ...
    └── ...

2. The basic modules
    GrADim is the main module for the library where callable submodules used for automatic differentation will be stored. Test will contain testing submodules for GrADim.

3. Test Suite
    We will use both TravisCI and CodeCov. Travis CI will be used to detect when a commit has been made and pushed onto Github, and then try to build the project and run tests each time it happens. Codecov will be used to provide an assessment of the code coverage.

4. Package distribution
    PyPi will be used to distribute the package based on the format in the directory tree.

5. Framework considerations
    Framework is not currently considered as current implementation is not as complicated. Should the implementation complexity evolves, we will then consider implementing a framework.

# 5. Implementation

We will now go into some implementation considerations

### 1. Data Structures
We will use floats if the user asks for a single output. If the user asks for a number of outputs for several inputs, we will use arrays. Different classes are defined for different input options as explained below.

### 2. Classes

We will have one class for the generic type of the automatic differentiation object.

Inside this generic class, we have one method for calculating derivative values, including a number of if-else statements:
- For the function input, we have one if-else block to check whether it contains matrices. If yes, the program will call the method of matrix operations for differentiation. Otherwise, call the usual automatic differentiation method.
- For the number of input variables, we have one if-else block to check if it is larger than 1. If univariate, the program will implement the differentiation function with only one variable. If multivariate, the program calls the multivariate differentiation method.
- For univariate differentiation, we have one nested if-else block to check whether the input variable is a single value or an array of values. If input is a single number, the program will implement simple differentiation. Otherwise, the input is an array, then the program will iterate through the array of values for simple differentiation.
- For multivariate differentiation, we have a nested if-else block to check whether the input variable is a matrix of  values. If it is a vector, the program will implement multivariate automatic differentiation. Otherwise, the input values are in matrix form, then the program will iterate through each vector and call the multivariate automatic differentiation.
- For the function implemented, we have one if-else block to check if the function contains matrices. If it contains matrices, the program will implement the matrix version of differentiation, in univariate or multivariate form, depending on the number of input variables. Otherwise, the program will implement the usual form of differentiation that do not invlove matrix multiplication.

For automatic differentiation, an elementary operation would be something like:

In [ ]:
def sin(self, x):
    if x is a vector:
        for variable in vector:
            self.partial_variable = self.cos(x)
            self.val = self.sin(x)

    else:
        self.der = self.cos(x)
        self.val = self.sin(x)

We have one subclass of differentiation that implements the most basic form of differentiation: input has one variable and one function, and the output is one value representing the function derivative. 
We have one subclass of differentiation, partial differentiation, that handles the input function with more than one variables.
We have one subclass of differentiation for automatic differentiation that covers the case where the input function is in matrix form.

Methods that all classes share include hard coding the basic differentiations, (e.g., $\frac{dc}{dx}$ = 0 where c is a constant, $\frac{d sin(x)}{dx}$ = cos(x), $\frac{dx^a}{dx}$ = ax<sup>a-1</sup>etc.) and chain rule. For multivariate differentiation, methods are defined to calculate partial derivatives ($\frac{\partial (xy)}{\partial x} = y, \frac{\partial (xy)}{\partial y} = x$). When necessary, we will overwrite the pre-defined methods so that the program would do the differentiation.

Name attributes contain function values and derivatives.

### 3. External dependencies
We will use numpy elementary operations (e.g., sin, cos, log, exp, tan, sinh, cosh, tanh, arcsin, arctan, etc.). We will use scipy is used mainly for more complicated matrix algebra. If the input function is passed in via scipy, we may use a scipy dependency in implementation.

### 4. Other considerations 
We will consider differentiation in polar coordinate as well as that in cartesian coordinate.

# 6. Licensing 

We would want our library to be open source and accessible to everyone. We would be okay with others mmodifying our code and if the modifications are distributed, and if it is used in comercial softwares, with proper acknowledgement. So we opt for MIT copyright license. 

- 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=077351ea-f043-41bb-a867-a9ce6280de54' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>